# Clifford+RZ and Hardware Compilation

You need the `pyqrack` package to run this notebook. [`vm6502q/pyqrack`](https://github.com/vm6502q/pyqrack) is a pure Python wrapper on the [`vm6502q/qrack`](https://github.com/vm6502q/qrack) quantum computer simulation framework core library. The preferred method of installation is from source code, at those GitHub repositories, but a package with default build precompiled binaries is available on [pypi](https://pypi.org/project/pyqrack/0.2.0/).

In [1]:
# For example, if your Jupyter installation uses pip:
# import sys
# !{sys.executable} -m pip install --platform=manylinux2014_x86_64 --only-binary=:all: pyqrack

[`QrackSimulator`](https://github.com/vm6502q/pyqrack/blob/main/pyqrack/qrack_simulator.py) is the "workhorse" of the `pyqrack` package. It instantiates simulated "registers" of qubits that we can act basic quantum gates between, to form arbitrary universal quantum circuits.

[`QrackCircuit`](https://github.com/vm6502q/pyqrack/blob/main/pyqrack/qrack_circuit.py) is an optional class for optimizing compilation. With it, one can define a circuit in advance, which is optimized upon definition. Then, one can save the optimized result to a file and later load it into a new or existing `QrackCircuit` instance. Ultimately, the circuit is executed by calling `run()` on a `QrackCircuit`, with a parameter of `QrackSimulator` of appropriate size. (The necessary width of the `QrackSimulator` can be determined with `QrackCircuit.get_qubit_count()`.)

In [2]:
import collections
import copy
import math
import os
import random
# import warnings

import numpy as np

import tensorcircuit as tc
import tensorcircuit.compiler.simple_compiler as tcsc

from qiskit import QuantumCircuit
from qiskit.compiler import transpile

from pyqrack import QrackSimulator, QrackCircuit

# If we disable OpenCL and set the max CPU qubits to max integer, we bypass qubit widths limits.
os.environ["QRACK_MAX_CPU_QB"]="-1"

In [3]:
width = 4

We're going to try to optimize the compilation of a quantum volume circuit for minimum gate count and depth of critical path.

In [4]:
def rand_1qb(circ, q):
    th = random.uniform(0, 4 * math.pi)
    ph = random.uniform(0, 4 * math.pi)
    lm = random.uniform(0, 4 * math.pi)
    circ.u(th, ph, lm, q)


def random_circuit(width, circ):
    for n in range(width):
        # Single bit gates
        for j in range(width):
            rand_1qb(circ, j)

        # Multi bit gates
        bit_set = [i for i in range(width)]
        while len(bit_set) > 1:
            b1 = random.choice(bit_set)
            bit_set.remove(b1)
            b2 = random.choice(bit_set)
            bit_set.remove(b2)
            circ.cx(b1, b2)

    return circ

In [5]:
orig_circ = random_circuit(width, QuantumCircuit(width))

print("Raw gate count: ", sum(dict(orig_circ.count_ops()).values()))
print("Raw depth of critical path: ", orig_circ.depth())
print("Raw qubit width: ", orig_circ.width())

Raw gate count:  24
Raw depth of critical path:  8
Raw qubit width:  4


If we have Qiskit and numpy installed, we can transpile to a near-Clifford Qiskit circuit.

In [6]:
basis_gates=["rz", "x", "y", "z", "sx", "sy", "s", "sdg", "cx", "cy", "cz", "swap", "iswap", "iswap_dg"]
circ = transpile(orig_circ, basis_gates=basis_gates, optimization_level=3)

print("Raw (near-Clifford) gate count: ", sum(dict(circ.count_ops()).values()))
print("Raw (near-Clifford) depth of critical path: ", circ.depth())
print("Raw (near-Clifford) qubit width: ", circ.width())

Raw (near-Clifford) gate count:  82
Raw (near-Clifford) depth of critical path:  24
Raw (near-Clifford) qubit width:  4


From there, we can simulate with Qrack's near-Clifford-optimized simulation layer, everything except for measurement sampling. We write the resulting "intermediate representation" into a file on disk.

In [7]:
qsim = QrackSimulator(width, isSchmidtDecomposeMulti=False, isSchmidtDecompose=False, isOpenCL=False)
qsim.run_qiskit_circuit(circ, 0)
qsim.out_to_file("qrack_circuit.chp")
qsim.reset_all()
qsim = None

Then, we read this into a new Qiskit circuit, which is still near-Clifford. The gate count and qubit width might be higher, but the circuit is entirely Clifford group, except for a single terminal layer of non-Clifford gates, (followed by either post selection or a rudimentary error-correction routine).

In using the `QrackSimulator.file_to_optimized_qiskit_circuit(filename)` method, we further remove the ancilla qubits, "injecting" their "magic states," bringing the circuit back into a general form, with Clifford and `U` gates.

In [8]:
circ = QrackSimulator.file_to_optimized_qiskit_circuit("qrack_circuit.chp")

if circ.width() <= 16:
    print(circ)
print("(Near-Clifford) gate count: ", sum(dict(circ.count_ops()).values()))
print("(Near-Clifford) depth of critical path: ", circ.depth())
print("(Near-Clifford) qubit width: ", circ.width())

            ┌──────────────┐                 ┌──────────────────┐           »
q_0: ───────┤ U(π/2,π/2,0) ├────────■────────┤ U(π/2,-π/2,-π/2) ├────────■──»
     ┌──────┴──────────────┴─────┐  │        └──────────────────┘        │  »
q_1: ┤ U(0.94631,-1.5084,1.4644) ├──┼────────────────────────────────────┼──»
     └┬──────────────────────────┤  │                                    │  »
q_2: ─┤ U(2.2804,-1.7306,1.3283) ├──┼────────────────────────────────────┼──»
      └──┬────────────────────┬──┘┌─┴─┐┌──────────────────────────────┐┌─┴─┐»
q_3: ────┤ U(0.72971,-π/2,-π) ├───┤ X ├┤ U(0.98175,-0.64053,-0.64053) ├┤ X ├»
         └────────────────────┘   └───┘└──────────────────────────────┘└───┘»
«           ┌──────────────┐           ┌─────────────────────────┐     »
«q_0: ──────┤ U(π/2,0,π/2) ├────────■──┤ U(1.1736,1.2769,2.8557) ├─────»
«           └──────────────┘        │  └─────────────────────────┘     »
«q_1: ──────────────────────────────┼──────────────────────────────────»
«     

If only terminal gates on wires are non-Clifford, then it's possible to efficiently simulate and sample the measurement distribution of this circuit.

In [9]:
def is_clifford_param(param):
    return np.isclose(param, 0) or np.isclose(param, math.pi / 2) or np.isclose(param, math.pi) or np.isclose(param, 3 * math.pi / 2) or np.isclose(param, -math.pi / 2) or np.isclose(param, -math.pi) or np.isclose(param, -3 * math.pi / 2)

efficient_circ = circ
is_classically_efficient = circ.width() == width
if is_classically_efficient:
    basis_gates = ["u", "cx", "cy", "cz", "swap", "iswap", "iswap_dg"]
    efficient_circ = transpile(efficient_circ, basis_gates=basis_gates, optimization_level=3)
    is_blocked = circ.width() * [False]
    for i in range(circ.width()):
        for j in range(len(circ.data)):
            op = circ.data[j].operation
            qubits = circ.data[j].qubits
            q1 = circ.find_bit(qubits[0])[0]
            if (len(qubits) < 2) and (q1 == i):
                was_blocked = is_blocked[i]
                if was_blocked and len(op.params) == 0:
                    is_classically_efficient = False
                    break
                for param in op.params:
                    if not is_clifford_param(param):
                        is_blocked[i] = True
                        if was_blocked:
                            is_classically_efficient = False
                            break

            if len(qubits) < 2:
                continue

            q2 = circ.find_bit(qubits[1])[0]
            if (q2 == i) and is_blocked[i]:
                is_classically_efficient = False
                break

        if not is_classically_efficient:
            break

if is_classically_efficient:
    print("Intermediate representation has only terminal non-Clifford gates; efficient classical simulation should be possible (with \"efficient_circ\" variable).")

`QrackCircuit` might reduce the gate count and depth further, but it's probably an unrealistic gate set for most gate-based quantum hardware. However, if we're using conventional Qrack simulation, rather than near-Clifford, this format is advantageous for native Qrack.

In [10]:
qcircuit = QrackCircuit.in_from_qiskit_circuit(circ)
qcircuit.out_to_file("qrack_circuit.qc")
qrack_circ = QrackCircuit.file_to_qiskit_circuit("qrack_circuit.qc")

if qrack_circ.width() <= 16:
    print(qrack_circ)
print("QrackCircuit gate count: ", sum(dict(qrack_circ.count_ops()).values()))
print("QrackCircuit depth of critical path: ", qrack_circ.depth())
print("QrackCircuit qubit width: ", qrack_circ.width())

     ┌─────────────┐┌──────────────┐┌─────────────┐┌──────────────┐»
q_0: ┤ Multiplexer ├┤1             ├┤ Multiplexer ├┤1             ├»
     ├─────────────┤│              │└─────────────┘│              │»
q_1: ┤ Multiplexer ├┤              ├───────────────┤              ├»
     ├─────────────┤│  Multiplexer │               │  Multiplexer │»
q_2: ┤ Multiplexer ├┤              ├───────────────┤              ├»
     ├─────────────┤│              │               │              │»
q_3: ┤ Multiplexer ├┤0             ├───────────────┤0             ├»
     └─────────────┘└──────────────┘               └──────────────┘»
«     ┌─────────────┐┌──────────────┐┌─────────────┐                 »
«q_0: ┤ Multiplexer ├┤1             ├┤ Multiplexer ├─────────────────»
«     └─────────────┘│              │└─────────────┘ ┌──────────────┐»
«q_1: ───────────────┤              ├────────────────┤1             ├»
«                    │  Multiplexer │┌──────────────┐│              │»
«q_2: ───────────────┤  

We might further optimize with tensor network software.

In [11]:
tc.set_backend("tensorflow")
tc.set_contractor("auto")
tc.set_dtype("complex64")

net = tc.Circuit.from_qiskit(circ)
net = tcsc.simple_compile(net)[0]
net.draw()

┌─────────────┐ ┌───┐ ┌────────────┐┌───┐ ┌────────────┐      »
q_0: ┤ Rz(-1.5708) ├─┤ H ├─┤ Rz(1.5708) ├┤ H ├─┤ Rz(3.1416) ├───■──»
     ├─────────────┴┐├───┤┌┴────────────┤├───┤┌┴────────────┤   │  »
q_1: ┤ Rz(-0.10639) ├┤ H ├┤ Rz(0.94631) ├┤ H ├┤ Rz(0.06236) ├───┼──»
     ├──────────────┤├───┤└┬────────────┤├───┤├─────────────┴┐  │  »
q_2: ┤ Rz(-0.24249) ├┤ H ├─┤ Rz(2.2804) ├┤ H ├┤ Rz(-0.15979) ├──┼──»
     ├─────────────┬┘├───┤┌┴────────────┤├───┤└──────────────┘┌─┴─┐»
q_3: ┤ Rz(-4.7124) ├─┤ H ├┤ Rz(0.72971) ├┤ H ├────────────────┤ X ├»
     └─────────────┘ └───┘└─────────────┘└───┘                └───┘»
«     ┌─────────────┐┌───┐ ┌────────────┐┌───┐                    »
«q_0: ┤ Rz(-3.1416) ├┤ H ├─┤ Rz(1.5708) ├┤ H ├─────────────────■──»
«     └─────────────┘└───┘ └────────────┘└───┘                 │  »
«q_1: ─────────────────────────────────────────────────────────┼──»
«                                                              │  »
«q_2: ─────────────────────────────────────────────────────────┼──»
«     ┌─────────────┐┌───┐┌─────────────┐┌───┐┌─────────────┐┌─┴─┐»
«q_3: ┤ Rz(-2.2113) ├┤ H ├┤ Rz(0.98175) ├┤ H ├┤ Rz(0.93027) ├┤ X ├»
«     └─────────────┘└───┘└─────────────┘└───┘└─────────────┘└───┘»
«         ┌───┐     ┌────────────┐    ┌───┐     ┌────────────┐                »
«q_0: ────┤ H ├─────┤ Rz(1.5708) ├────┤ H ├─────┤ Rz(1.5708) ├────────────────»
«         └───┘     └────────────┘    └───┘     └────────────┘                »
«q_1: ────────────────────────────────────────────────────────────────────────»
«                                                                             »
«q_2: ────────────────────────────────────────────────────────────────────────»
«     ┌────────────┐    ┌───┐     ┌────────────┐    ┌───┐     ┌──────────────┐»
«q_3: ┤ Rz(1.0331) ├────┤ H ├─────┤ Rz(2.0908) ├────┤ H ├─────┤ Rz(-0.87604) ├»
«     └────────────┘    └───┘     └────────────┘    └───┘     └──────────────┘»
«           ┌────────────┐┌───┐┌────────────┐┌───┐ ┌────────────┐      »
«q_0: ──■───┤ Rz(1.2849) ├┤ H ├┤ Rz(1.1736) ├┤ H ├─┤ Rz(2.8477) ├──────»
«       │   └────────────┘└───┘└────────────┘└───┘ └────────────┘      »
«q_1: ──┼──────────────────────────────────────────────────────────────»
«       │                                                              »
«q_2: ──┼───────────────────────────────────────────────────────────■──»
«     ┌─┴─┐┌─────────────┐┌───┐┌────────────┐┌───┐┌──────────────┐┌─┴─┐»
«q_3: ┤ X ├┤ Rz(0.99865) ├┤ H ├┤ Rz(1.1394) ├┤ H ├┤ Rz(-0.89424) ├┤ X ├»
«     └───┘└─────────────┘└───┘└────────────┘└───┘└──────────────┘└───┘»
«                                                                »
«q_0: ───────────────────────────────────────────────────────────»
«                                                                »
«q_1: ────────────────────────────────────────────────────────■──»
«     ┌─────────────┐┌───┐┌─────────────┐┌───┐ ┌──────────┐   │  »
«q_2: ┤ Rz(-2.9093) ├┤ H ├┤ Rz(0.91819) ├┤ H ├─┤ Rz(2.21) ├───┼──»
«     ├─────────────┤├───┤└┬────────────┤├───┤┌┴──────────┴┐┌─┴─┐»
«q_3: ┤ Rz(-4.6589) ├┤ H ├─┤ Rz(1.5111) ├┤ H ├┤ Rz(2.4103) ├┤ X ├»
«     └─────────────┘└───┘ └────────────┘└───┘└────────────┘└───┘»
«                                                                »
«q_0: ────────────────────────────────────────────────────────■──»
«     ┌─────────────┐┌───┐┌─────────────┐┌───┐┌────────────┐  │  »
«q_1: ┤ Rz(-4.4715) ├┤ H ├┤ Rz(0.43362) ├┤ H ├┤ Rz(4.2146) ├──┼──»
«     └─────────────┘└───┘└─────────────┘└───┘└────────────┘  │  »
«q_2: ────────────────────────────────────────────────────────┼──»
«      ┌────────────┐┌───┐ ┌────────────┐┌───┐┌───────────┐ ┌─┴─┐»
«q_3: ─┤ Rz(-2.516) ├┤ H ├─┤ Rz(1.6357) ├┤ H ├┤ Rz(2.507) ├─┤ X ├»
«      └────────────┘└───┘ └────────────┘└───┘└───────────┘ └───┘»
«     ┌─────────────┐┌───┐┌────────────┐┌───┐┌────────────┐        »
«q_0: ┤ Rz(-3.7657) ├┤ H ├┤ Rz(1.3831) ├┤ H ├┤ Rz(3.1416) ├─X───■──»
«     └─────────────┘└───┘└────────────┘└───┘└─────

We can sample with tensorcircuit at this point, or we can try another round of optimization.

In [12]:
%%time
print(net.sample(allow_state=True, batch=256, format="count_dict_bin"))

{'0000': 11, '0001': 12, '0010': 2, '0011': 1, '0100': 6, '0101': 4, '0110': 20, '0111': 6, '1000': 84, '1001': 6, '1010': 7, '1011': 37, '1100': 7, '1101': 3, '1110': 20, '1111': 30}
CPU times: user 89 ms, sys: 286 µs, total: 89.3 ms
Wall time: 87.5 ms


In [13]:
circ = net.to_qiskit()
basis_gates=["rz", "x", "y", "z", "sx", "sy", "s", "sdg", "cx", "cy", "cz", "swap", "iswap", "iswap_dg"]
circ = transpile(circ, basis_gates=basis_gates, optimization_level=3)

In [14]:
def cross_entropy(measurement_list, qiskit_circuit):
    perm_count = 1 << qiskit_circuit.width()
    ideal_shots = perm_count << 12

    sim = QrackSimulator(qubitCount = qiskit_circuit.width(), isStabilizerHybrid=False)
    sim.run_qiskit_circuit(qiskit_circuit, 0)
    ideal_result = sim.measure_shots(list(range(qiskit_circuit.width())), ideal_shots)
    ideal_result = dict(collections.Counter(ideal_result))
    for key, value in ideal_result.items():
        ideal_result[key] = value / ideal_shots
        
    ideal_probs_sv = []
    for i in range(perm_count):
        ideal_probs_sv.append(ideal_result[i] if i in ideal_result else 0)
    
    ideal_probs = []
    for bit_string in measurement_list:
        ideal_probs.append(ideal_probs_sv[bit_string])
                               
    return (perm_count * np.mean(ideal_probs)) - 1


def hellinger_fidelity(measurement_list, qiskit_circuit):
    perm_count = 1 << qiskit_circuit.width()
    ideal_shots = perm_count << 12

    t_histogram = collections.Counter(measurement_list)
    shot_count = sum(t_histogram.values())
    t_histogram = dict(t_histogram)
        
    histogram = {}
    for key, value in t_histogram.items():
        histogram[key] = value / shot_count

    sim = QrackSimulator(qubitCount = qiskit_circuit.width(), isStabilizerHybrid=False)
    sim.run_qiskit_circuit(qiskit_circuit, 0)
    ideal_result = sim.measure_shots(list(range(qiskit_circuit.width())), ideal_shots)
    ideal_result = dict(collections.Counter(ideal_result))
    for key, value in ideal_result.items():
        ideal_result[key] = value / ideal_shots
        
    ideal_probs_sv = []
    for i in range(perm_count):
        ideal_probs_sv.append(ideal_result[i] if i in ideal_result else 0)

    fidelity = 0
    for qubit_permutation in histogram.keys():
        ideal_prob = ideal_result[qubit_permutation] if qubit_permutation in ideal_result else 0
        normalized_frequency = histogram[qubit_permutation]
        fidelity += math.sqrt(ideal_prob * ideal_prob)
    fidelity *= fidelity
                               
    return fidelity



if width <= 16:
    # The circuit has 0 ancillae, so we can generate samples with conventional simulation.
    qsim = QrackSimulator()
    qcircuit.run(qsim)
    for i in range(width, qcircuit.get_qubit_count()):
        qsim.force_m(i, False)
    shots = 1 << (width + 12)
    samples = qsim.measure_shots(list(range(width)), shots)
    # print("XEB fidelity: ", cross_entropy(samples, orig_circ))
    print("Hellinger fidelity: ", hellinger_fidelity(samples, orig_circ))

Device #0, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_Intel(R)_UHD_Graphics_[0x9bc4].ir
Device #1, Loaded binary from: /home/iamu/.qrack/qrack_ocl_dev_NVIDIA_GeForce_RTX_3080_Laptop_GPU.ir
Hellinger fidelity:  1.0


In [15]:
if is_classically_efficient:
    # Qrack needs debugging, for now:

    # The circuit has 0 ancillae and only a terminal layer of non-Clifford gates, so we can generate samples with "hybrid stabilizer" simulation.
    # qsim = QrackSimulator(qubitCount = efficient_circ.width(), isSchmidtDecomposeMulti = False, isSchmidtDecompose = False, isOpenCL=False)
    # qsim.run_qiskit_circuit(efficient_circ, 0)
    # print(qsim.measure_shots(list(range(width)), shots))